In [1]:
# ref: https://github.com/moskomule/senet.pytorch/blob/master/senet/se_module.py

from torch import nn


class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

In [ ]:
# ref: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
# ref: https://hoya012.github.io/blog/DenseNet-Tutorial-2/

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

# load dataset
trainset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=False,
                                       download=True, transform=transform)

# set transform
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

# train/valid split
validation_ratio= 0.1
random_seed = 10

num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(validation_ratio * num_train))

np.random.seed(random_seed)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

#set loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          sampler=train_sampler, num_workers=2)
validloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          sampler=valid_sampler, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.se1 = SELayer(6, 1)

        self.conv2 = nn.Conv2d(6, 16, 5)
        self.se2 = SELayer(16, 2)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x= self.se1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.se2(x)
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

net = Net().to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01, weight_decay=0.0001)

exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

Files already downloaded and verified
Files already downloaded and verified
cuda:0


In [ ]:
best_valid_acc=0
for epoch in range(50):  # loop over the dataset multiple times
    
    net.train()
    ### learnint rate scheduling ###
    exp_lr_scheduler.step()
    
    running_loss = 0
    total = 0
    correct = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        # for train acc
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    

    ### train loss/acc ###
    print('[%d] loss: %.3f, train_acc: %.3f %%, lr:%f' %
          (epoch + 1, running_loss / 50000, 100.*correct/total, exp_lr_scheduler.get_lr()[0]))
    
    net.eval()
    ### vlaid acc ###
    correct = 0
    total = 0
    with torch.no_grad():
        for data in validloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # save early stopping point (best valid accuracy)
    valid_acc= 100.*correct/total
    if best_valid_acc<valid_acc:
        best_valid_acc= valid_acc
        torch.save(net.state_dict(), "./save_best.pth") #save
    print('[%d] valid_acc: %.3f %%, best: %.3f %%' %(epoch + 1, valid_acc, best_valid_acc))
    
    


print('Finished Training')

[1] loss: 0.012, train_acc: 37.204 %, lr:0.010000
[1] valid_acc: 46.340 %, best: 46.340 %
[2] loss: 0.010, train_acc: 48.942 %, lr:0.010000
[2] valid_acc: 52.040 %, best: 52.040 %
[3] loss: 0.009, train_acc: 52.673 %, lr:0.010000
[3] valid_acc: 51.700 %, best: 52.040 %
[4] loss: 0.009, train_acc: 54.467 %, lr:0.010000
[4] valid_acc: 53.960 %, best: 53.960 %
[5] loss: 0.009, train_acc: 55.991 %, lr:0.010000
[5] valid_acc: 53.080 %, best: 53.960 %
[6] loss: 0.008, train_acc: 57.236 %, lr:0.010000
[6] valid_acc: 55.740 %, best: 55.740 %
[7] loss: 0.008, train_acc: 58.127 %, lr:0.010000
[7] valid_acc: 56.160 %, best: 56.160 %
[8] loss: 0.008, train_acc: 58.707 %, lr:0.010000
[8] valid_acc: 57.520 %, best: 57.520 %
[9] loss: 0.008, train_acc: 59.347 %, lr:0.010000
[9] valid_acc: 57.980 %, best: 57.980 %
[10] loss: 0.008, train_acc: 60.087 %, lr:0.010000
[10] valid_acc: 55.520 %, best: 57.980 %
[11] loss: 0.008, train_acc: 60.700 %, lr:0.010000
[11] valid_acc: 58.480 %, best: 58.480 %
[12] l

In [ ]:
# load & eval

load_model = Net().to(device)
load_model.load_state_dict(torch.load("./save_best.pth"))

load_model.eval()

### test acc ###
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = load_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc= 100.*correct/total

print('test_acc: %.3f %%' %(test_acc))


test_acc: 62.440 %
